# Load all seasons and insert to postgresql database

### It also handles stage unique end dates, which is critical to derive league table from fixtures

## Import

In [68]:
import pandas as pd
import logging
from os import listdir
import os.path
from os.path import isfile, join
import psycopg2
from sqlalchemy import create_engine
import numpy as np

## Params

In [69]:
user = 'postgres'
password = 'postgres'
databasename = 'soccer-db'
host = 'localhost'
port = '5432'

root_path = 'L:/Dev/Sandbox/Apps Development/SoccerStats/historical result files/France'

# Only ligue1 has been treated so far
csv_root_path_ligue1 = os.path.join(root_path,'ligue1')

# Here we could add other championship links...
# To do

## Logging

In [70]:
logger = logging.getLogger('myAppDbInsert')
hdlr = logging.FileHandler(os.path.join(root_path,'ligue1comDbInsertion.log'))
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.DEBUG)

# close log file objects
#logger.removeHandler(hdlr)
#del logger,hdlr

## Functions

### [db_connect] connect to postgres db and give back conn object

In [71]:
def db_connect():
    try:
        conn = psycopg2.connect("dbname='{0}' user='{1}' host='{2}' password='{3}'".format(databasename, user, host, password))
    except:
        print("I am unable to connect to the database")

    # Open a cursor to perform database operations
    return conn

### [db_seasons_table_to_dic] convert seasons table to dictionary {name:id}

In [72]:
def db_seasons_table_to_dic(cur):
    cur.execute("select * from seasons")
    rows = cur.fetchall()
    dic = {}
    for row in rows:
        dic[row[1]] = row[0]
    return dic

### [db_allcompetitions_table_to_dic] convert allcompetitions table to dictionary {name:id}

In [73]:
def db_allcompetitions_table_to_dic(cur):
    cur.execute("select * from allcompetitions")
    rows = cur.fetchall()
    dic = {}
    for row in rows:
        dic[row[1]] = row[0]
    return dic

### [db_competitions_table_to_dic] convert competitions table to dictionary {name:id}

In [74]:
def db_competitions_table_to_dic(cur, competitionname):
    cur.execute("select * from competitionid('{0}')".format(competitionname))
    rows = cur.fetchall()
    dic = {}
    for row in rows:
        dic[row[0]] = row[1]
    return dic

### [db_stages_table_to_dic] convert stages table to dictionary {name:id}

In [75]:
def db_stages_table_to_dic(cur):
    cur.execute("select * from stages")
    rows = cur.fetchall()
    dic = {}
    for row in rows:
        dic[row[1]] = row[0]
    return dic

### [db_teams_table_to_dic] convert teams table to dictionary {name:id}

In [76]:
def db_teams_table_to_dic(cur):
    cur.execute("select * from teams")
    rows = cur.fetchall()
    dic = {}
    for row in rows:
        dic[row[1]] = row[0]
    return dic

### [db_fixturestatus_table_to_dic] convert fixturestatus table to dictionary {name:id}

In [77]:
def db_fixturestatus_table_to_dic(cur):
    cur.execute("select * from fixturestatus")
    rows = cur.fetchall()
    dic = {}
    for row in rows:
        dic[row[1]] = row[0]
    return dic

### [db_get_seasons_name] get list of seasons name

In [78]:
def db_get_seasons(cur):
    cur.execute("select name from seasons")
    rows = cur.fetchall()
    return [x[0] for x in rows]

### [db_get_stages_name] get list of stages name

In [79]:
def db_get_stages(cur):
    cur.execute("select name from stages")
    rows = cur.fetchall()
    return [x[0] for x in rows]

### [db_get_fixtures_max_index] return max index from fixtures table

In [80]:
def db_get_fixtures_max_index(cur):
    cur.execute("select max(id) from fixtures")
    rows = cur.fetchall()
    if len(rows) != 1:
        # there should be exactly one row of result, a tuple (x,y) actually
        return -1
    # this is a tuple (x,y)
    res = rows[0]
    
    if res[0] == None:
        # tuple is (None,) -> table must be empty
        return 1
    
    # table not empty, return the actual max index
    return res[0]

### [db_week_dates_to_df] call to week_dates database function, which return dates associated to each stage of type "week"

In [81]:
def db_week_dates_to_df(cur, competition, season):
    cur.execute("select * from week_dates('{0}', '{1}')".format(competition, season))
    colnames = [desc[0] for desc in cur.description]
    
    df = pd.DataFrame(data=cur.fetchall(), columns=colnames)
    
    # is there an override define for this season?
    key = competition.replace(' ','').lower() + '_' + season.replace('/','_')
    df = handle_week_override(df, key)
    
    return df

### [get_delayed_fixtures_to_df] get delayed fixtures for given competition/season, returns dataframe

In [98]:
# override is a dictionary defining a list of week stage to substitute for each competition/season
# for instance: {'Ligue1_2016/2017': [('Week 11', 'Week 14')]}
# means 'Week 11' is currently wrongly place after 'Week 14'
# we need to decrement all weekid from Week 12 to week 14, and increment 'Week 11' by 3 in that case (14-11)
# that's useful for messed up dates
ovrd = {
    'ligue1_1970_1971': [('Week 11', 'Week 14')],
    'ligue1_1972_1973': [('Week 17', 'Week 29')],
    'ligue1_1977_1978': [('Week 15', 'Week 17')],
    'ligue1_1984_1985': [('Week 13', 'Week 14'), ('Week 13', 'Week 14'), ('Week 22', 'Week 27')],
    'ligue1_2009_2010': [('Week 11', 'Week 17')]
}

# substitute week1 and week2 in dataframe, supposing week 2 is currently wrongly after week 3
def substitute_weeks_in_df(df, week1, week2):
    
    # get week ids
    i1 = int(week1[-2:])
    i2 = int(week2[-2:])
    diff = i2 - i1
    
    # dummy values used for replacement
    tmp = 'Week XX'
    tmp_start = 'Week SS'
    
    # first, we replace the troublesome week id with a dummy value to keep track of it later
    df['stage'].replace(['Week ' + "%02d" % (i1,)], [tmp_start], inplace=True)
    
    for i in range(i1 + 1, i2 + 1):
        
        curr_week = 'Week ' + "%02d" % (i,)
        # we want to decrement week id here:
        new_week = 'Week ' + "%02d" % (i - 1,)
        
        # actual replacement here:
        df['stage'].replace([curr_week], [tmp], inplace=True)
        df['stage'].replace([new_week], [curr_week], inplace=True)
        df['stage'].replace([tmp], [new_week], inplace=True)
    
    
    # finally, we increment week id by the diff
    df['stage'].replace([tmp_start], ['Week ' + "%02d" % (i1 + diff,)], inplace=True)
    
    return df

def handle_week_override(df, key):
    if(key in ovrd):
        for tuple in ovrd[key]:
            df = substitute_weeks_in_df(df, tuple[0], tuple[1])
    return df

def get_delayed_fixtures_to_df(cur, competition, season):
    df = db_week_dates_to_df(cur, competition, season)
    df2 = pd.DataFrame(columns=df.columns)
    current_stage_id = 1
    next_stage_id = 2
    i = 0
    j = 1
    for stage in df['stage'][1:]:
        stage_id = int(stage[-2:])
        if(stage_id != current_stage_id and stage_id != next_stage_id):
            df2.loc[i] = df.loc[j]
            i = i+1
        elif(stage_id == next_stage_id):
            current_stage_id += 1
            next_stage_id += 1
        j = j+1
    return df2

### [get_week_unique_dates_to_df] get week unique start/end dates for given competition/season, returns dataframe

In [83]:
def get_week_unique_dates_to_df(cur, competition, season):
    df1 = get_delayed_fixtures_to_df(cur, competition, season)
    df2 = db_week_dates_to_df(cur, competition, season)
    df3 = df2.merge(df1, on=['season','competition','stage', 'date'], how='left')
    df4 = df3[df3.nb_games_y.isnull()][['stage','date']]
    df5 = df4.groupby(['stage'], sort=False)['date'].max()
    df6 = df4.groupby(['stage'], sort=False)['date'].min()
    return pd.DataFrame({'start_date': df6, 'end_date': df5}).reset_index()[['stage','start_date','end_date']]

### [process_seasons_week_unique_dates] process all seasons for week unique start/end dates insertion to database

In [84]:
def check_week_unique_dates_df(df):
    
    correct_len = [34, 37, 38, 40]
    
    # check df len
    if(df.shape[0] not in correct_len):
        logger.error("Dataframe length not expected: {0}".format(df.shape[0]))
        return False
    
    # verify weeks and dates are ordered ASC
    prev_end_date = df.iloc[0]['end_date']
    for i in range(1, df.shape[0]):
        curr_end_date = df.iloc[i]['end_date']
        
        # messed up dates check
        if(prev_end_date > df.iloc[i]['start_date'] or df.iloc[i]['start_date'] > curr_end_date):
            logger.error("Dataframe dates are messed up in row {0}, for start_date={1}".format(i, df.iloc[i]['start_date']))
            return False
        
        # big gap dates check (between previous and current week)
        day_diff = (prev_end_date - df.iloc[i]['start_date']).days
        if(day_diff > 30):
            logger.error("There is a big gap of {0} days between 2 weeks in the dataframe: start_date={1}, end_date={2}".format(day_diff, df.iloc[i]['start_date'], df.iloc[i]['end_date']))
            return False
        
        prev_end_date = curr_end_date
    return True

def process_seasons_week_unique_dates(competition):
    
    # connect to database
    conn = db_connect()
    cur = conn.cursor()
    
    # get seasons list
    seasons = db_get_seasons(cur)
    
    # get stages and competitions dic from db
    dic_stages = db_stages_table_to_dic(cur)
    dic_competitions = db_competitions_table_to_dic(cur, competition)
    
    for season in seasons:
        
        # get week unique dates df
        df = get_week_unique_dates_to_df(cur, competition, season)
        
        if(check_week_unique_dates_df(df)):
            
            # normalize
            df['competitionid'] = dic_competitions[season]
            df['stageid'] = [dic_stages[x] for x in df['stage']]
            df = df[['competitionid','stageid', 'start_date', 'end_date']]
            
            # insert week dates to database
            #db_insert_to_table(df, 'stage_dates')
            print('Week dates inserted to db for {0}, season {1}..'.format(competition, season))
            
        else:
            # log issue
            logger.error("Dataframe didn't pass the check for {0}, season {1}".format(competition, season))
            
    # close connection
    conn.close()
    

### [histo_csv_to_df] load all csv files and convert into pandas dataframe

In [85]:
def histo_csv_to_df(root_path):
    
    csvfiles = [f for f in listdir(root_path) if isfile(join(root_path, f)) and f.lower().endswith('.csv')]
    
    d = {}
    data = []
    firstfile = True
    column_names = []
    
    for csvfile in csvfiles:

        # open csv file
        fname = join(root_path, csvfile)
        with open(fname) as f:
            content = f.readlines()
        
        # only for first file
        if firstfile:
            column_names = [x.strip() for x in content[0].split(',')]
            firstfile = False
        
        # clean data
        content = [x.strip() for x in content]
        content = content[1:]

        # explode rows into list
        for item in content:
            data += [item.split(',')]

    return pd.DataFrame(columns=column_names, data=data)

### [get_competitions_to_df] construct competitions df out of fixtures df

In [86]:
def get_competitions_to_df(df_fixtures, dic_seasons, allcompetitionid):
    df = pd.DataFrame(df_fixtures.season.unique(), columns=['season'])
    df['seasonid'] = [dic_seasons[x] for x in df['season']]
    df['allcompetitionid'] = allcompetitionid
    return df[['seasonid', 'allcompetitionid']]

### [get_registered_teams] get registered teams for each competitions

In [87]:
def get_registered_teams(df_fixtures_normalized, dic_stages):
    
    cols = ['competitionid', 'teamid']
    df = pd.DataFrame({'competitionid':[],'teamid':[]},columns=cols, dtype=int)
    for competitionid in df_fixtures_normalized.competitionid.unique():
        
        # get only rows for this competitionid
        df_competitionid = df_fixtures_normalized.loc[df_fixtures_normalized['competitionid'] == competitionid]
        
        # remove playoffs if any
        df_competitionid = df_competitionid.loc[df_competitionid['stageid'] != dic_stages['Relegation Playoff']]
                
        # get unique teamid among hometeamdid and awayteamid cols
        unique_teamids = np.unique(df_competitionid[['hometeamid','awayteamid']])
        
        # create a new dataframe to wrap it up
        new_df = pd.DataFrame(columns=cols)
        new_df['teamid'] = [x for x in unique_teamids]
        new_df['competitionid'] = competitionid
        df = df.append(new_df)
        
    return df

### [analyse_histo] analyse dataframe resulting from preceding function

In [88]:
def analyse_histo(df):
    
    # get distinct seasons
    seasons = list(df.season.unique())
    
    # remove playoff from the dataframe
    df_no_playoff = df.loc[df['week'] != 'Play-off']
    
    # for each season
    for season in seasons:
        
        # filter on season
        df_tmp = df_no_playoff.loc[df['season'] == season]
        
        # get distinct # teams
        nb_team = len(df_tmp.team_home.unique())
        
        # estimate # games
        nb_game_estim = (nb_team -1) * 2 * nb_team / 2
        
        # get actual # games from dataframe and check it matches estimation
        nb_game = len(df_tmp)
        
        # print results
        diff = nb_game - nb_game_estim
        status = diff == 0
        print("Season {0}: nb_game_estim = {1}, nb_game = {2}, diff = {3}, status: {4}".format(season,nb_game_estim, nb_game, diff,status))
    

### [normalize_data] normalize dataframe to get a clean dataset before insertion to database

In [89]:
# we need to find a much better way to handle all that (using unicode)
def handle_accent(s):
    s = s.replace("ã¨", "e")
    s = s.replace("ã©", "e")
    s = s.replace("ã¯", "i")
    s = s.replace("ã", "a")
    s = s.replace("a«", "e")
    s = s.replace("a¢", "a")
    s = s.replace("a§", "c")
    s = s.replace("ê", "e")
    s = s.replace("î", "i")
    s = s.replace("â", "a")
    return s

In [90]:
def format_team(team):
    team = handle_accent(team)
    if team == "LOSC":
        return "Lille OSC"
    elif team == "GF38":
        return "Grenoble Foot 38"
    else:
        return team

In [91]:
def format_stage(stage):
    if stage[:4] == 'Week':
        id = stage[4:]
        if len(id) == 1:
            id = '0' + id
        return 'Week ' + id
    elif stage == 'Play-off':
        return 'Relegation Playoff'
    return stage

In [92]:
def normalize_data(df_fixtures, dic_competitions, dic_stages, dic_fixturestatus, fixturestatus, dic_teams, fixtures_max_index):
    
    # create competitionid column from season name and dic competitions {seasonname:competitionid}
    df_fixtures['competitionid'] = [dic_competitions[x] for x in df_fixtures['season']]
    
    # format stages to match database format
    df_fixtures['stage'] = [format_stage(x) for x in df_fixtures['week']]
    
    # replace stages by their ids
    df_fixtures['stageid'] = [dic_stages[x] for x in df_fixtures['stage']]
    
    # replace fixture status by their ids - all fixtures are already completed for that exercize
    df_fixtures['statusid'] = dic_fixturestatus[fixturestatus]
    
    # format teams
    df_fixtures['home_team_formated'] = [format_team(x) for x in df_fixtures['team_home']]
    df_fixtures['away_team_formated'] = [format_team(x) for x in df_fixtures['team_away']]
    
    # replace teams by their ids
    df_fixtures['hometeamid'] = [dic_teams[x] for x in df_fixtures['home_team_formated']]
    df_fixtures['awayteamid'] = [dic_teams[x] for x in df_fixtures['away_team_formated']]
    
    # create index table
    df_fixtures['id'] = [x for x in range(fixtures_max_index, fixtures_max_index + df_fixtures.shape[0])]
    df_fixtures = df_fixtures.set_index('id')    
    
    # rename columns to match database
    df_fixtures['date'] = df_fixtures['date']
    df_fixtures['homescore'] = df_fixtures['score_home']
    df_fixtures['awayscore'] = df_fixtures['score_away']
    
    # keep only a few columns
    return df_fixtures[['date','competitionid','stageid','statusid','hometeamid','awayteamid','homescore','awayscore']]

### [db_insert_to_table] insert dataframe to table

In [93]:
def db_insert_to_table(df, table,index_col=''):
    engine = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(user, password, host, port, databasename))
    if(index_col!=''):
        df.to_sql(table, engine, if_exists='append',index=index_col)
    else:
        df.to_sql(table, engine, if_exists='append',index=False)
    engine.dispose();

### [db_process_histo_insert] load all histo csv files into dataframe and insert it to database

In [94]:
def db_process_histo_insert(root_path, competitionname, fixturestatus):
    # load all games data from histo csv files into a dataframe
    df_fixtures = histo_csv_to_df(root_path)

    # connect to database
    conn = db_connect()
    cur = conn.cursor()
    
    # first we need to insert competitions
    dic_seasons = db_seasons_table_to_dic(cur)
    dic_allcompetitions = db_allcompetitions_table_to_dic(cur)
    df_competitions_to_insert = get_competitions_to_df(df_fixtures, dic_seasons, dic_allcompetitions[competitionname])
    
    # insert competitions to database
    db_insert_to_table(df_competitions_to_insert, 'competitions', 'id')
    print('competitions inserted to db..')
    
    # get other main tables info
    seasons = db_get_seasons(cur)
    stages = db_get_stages(cur)
    dic_competitions = db_competitions_table_to_dic(cur, competitionname)
    dic_stages = db_stages_table_to_dic(cur)
    dic_fixturestatus = db_fixturestatus_table_to_dic(cur)
    dic_teams = db_teams_table_to_dic(cur)
    fixtures_max_index = db_get_fixtures_max_index(cur)
    
    # close database
    conn.close()
    
    # normalize fixtures data
    df_normalized = normalize_data(df_fixtures, dic_competitions, dic_stages, dic_fixturestatus, fixturestatus, dic_teams, fixtures_max_index)
    
    # insert fixtures data to database
    db_insert_to_table(df_normalized, 'fixtures', 'id')
    print('{0} fixtures inserted to db..'.format(competitionname))
    
    # registered teams for each competitions
    df_registeredteams = get_registered_teams(df_normalized, dic_stages)
    
    # insert registered teams to database
    db_insert_to_table(df_registeredteams, 'registeredteams')
    print('{0} registered teams inserted to db..'.format(competitionname))

## Test API

In [95]:
## TEST DATABASE INSERTION ##
## LIGUE 1 ##
#db_process_histo_insert(csv_root_path_ligue1, 'Ligue 1', 'Completed')

competitions inserted to db..
Ligue 1 fixtures inserted to db..
Ligue 1 registered teams inserted to db..


In [99]:
## TEST week unique end dates ##
# connect to database
#conn = db_connect()
#cur = conn.cursor()
#competition = 'Ligue 1'
#season = '1988/1989'
#df = get_week_unique_dates_to_df(cur, competition, season)
#conn.close()

#process_seasons_week_unique_dates('Ligue 1')

# close log file objects
#logger.removeHandler(hdlr)
#del logger,hdlr

## Test dataframe shape

In [30]:
#dtypeCount =[df.iloc[:,i].apply(type).value_counts() for i in range(df.shape[1])]
#dtypeCount